## Using Spark GraphFrames to process BikeShare data

In [ ]:
import pandas as pd

In [ ]:
trips_filename = '../201508_trip_data.csv'
stations_filename = '../201508_station_data.csv'

trips_pdf = pd.read_csv(trips_filename)
stations_pdf = pd.read_csv(stations_filename)

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from graphframes import *

In [ ]:
session = SparkSession.builder.appName("BikeShareGF").getOrCreate()

In [ ]:
display(stations_pdf.head())

In [ ]:
# create nodes from stations.
#
# Vertex DataFrame: A vertex DataFrame should contain a special column named "id" 
# which specifies unique IDs for each vertex in the graph.
#
stations_pdf = stations_pdf.rename(columns={'station_id':'id'})
vertices = session.createDataFrame(stations_pdf)

In [ ]:
vertices.printSchema()

In [ ]:
display(trips_pdf.head())

In [ ]:
# pick our set of columns to represent the edges
edge_columns = ['Start Terminal', 'End Terminal', 'Trip ID', 'Start Date', 'End Date']

trips_edges_pdf = trips_pdf.loc[:,edge_columns]

In [ ]:
# rename the start and end terminal to 'src' and 'dst'
# An edge DataFrame should contain two special columns: "src" (source vertex ID of edge) 
# and "dst" (destination vertex ID of edge).
trips_edges_pdf = trips_edges_pdf.rename(columns={'Start Terminal':'src', 'End Terminal':'dst'})

In [ ]:
display(trips_edges_pdf.head())

In [ ]:
edges = session.createDataFrame(trips_edges_pdf)

In [ ]:
edges.printSchema()

In [ ]:
g = GraphFrame(vertices, edges)

In [ ]:
print(g.vertices.count())

In [ ]:
print(g.edges.count())

In [ ]:
g.inDegrees.sort('inDegree', ascending=False).show()

In [ ]:
motifs = g.find("(a)-[e]->(b); (b)-[e2]->(a)")

In [ ]:
display(motifs.count())

In [ ]:
# Find chains of 4 vertices.
chain4 = g.find("(a)-[ab]->(b); (b)-[bc]->(c); (c)-[cd]->(d)")

In [ ]:
chain4.count()